In [68]:
from sklearn.metrics.pairwise import cosine_similarity
import fiftyone as fo
import fiftyone.zoo as foz
from tqdm import tqdm
import numpy as np
import os

In [69]:
"""
Вспомогательный скрипт для определения наличия дубликатов в датасете.

-----------------
Входные параметры:
IMG_DIR - путь до папки с изображениями.

"""

'\nВспомогательный скрипт для определения наличия дубликатов в датасете.\n\n-----------------\nВходные параметры:\nIMG_DIR - путь до папки с изображениями.\n\n'

In [70]:
IMG_DIR = ""

In [71]:
dataset = fo.Dataset.from_images_dir(IMG_DIR)

 100% |███████████████| 2401/2401 [496.0ms elapsed, 0s remaining, 4.9K samples/s]      


In [72]:
model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")

In [73]:
"""
Получение имбедингов для дальнейшего сравнения изображений.
"""
embeddings = dataset.compute_embeddings(model)

print(embeddings.shape)

 100% |███████████████| 2401/2401 [1.0m elapsed, 0s remaining, 39.0 samples/s]      
(2401, 1280)


In [74]:
"""
Нахождение косинусного расстояния попарно между всеми изображениями (чем больше значение, тем больше схожесть).
"""
similarity_matrix = cosine_similarity(embeddings)

print(similarity_matrix.shape)
print(similarity_matrix)

(2401, 2401)
[[1.         0.66629581 0.31211735 ... 0.59349806 0.66387056 0.26144291]
 [0.66629581 1.         0.22304806 ... 0.47329166 0.51415623 0.19521839]
 [0.31211735 0.22304806 1.         ... 0.28636364 0.55591165 0.96977667]
 ...
 [0.59349806 0.47329166 0.28636364 ... 1.         0.50199478 0.23014394]
 [0.66387056 0.51415623 0.55591165 ... 0.50199478 1.         0.46038663]
 [0.26144291 0.19521839 0.96977667 ... 0.23014394 0.46038663 1.        ]]


In [75]:
"""
Исключение проверки схожести на самого себя.
"""
n = len(similarity_matrix)

similarity_matrix = similarity_matrix - np.identity(n)

In [76]:
id_map = [s.id for s in dataset.select_fields(["id"])]

In [77]:
"""
Нахождение дубликатов по пороговому значению.
"""
thresh = 0.96
samples_to_remove = set()
list_of_duplicates = []

for idx, sample in enumerate(dataset):
    if sample.id not in samples_to_remove:
        
        dup_idxs = np.where(similarity_matrix[idx] > thresh)[0]
        for dup in dup_idxs:
            samples_to_remove.add(id_map[dup])

        if len(dup_idxs) > 0:
            sample.tags.append("has_duplicates")
            sample.save()
            list_of_duplicates.append(sample.filepath)

    else:
        sample.tags.append("duplicate")
        sample.save()


In [78]:
print(len(list_of_duplicates))

284


In [79]:
"""
Удаление дубликатов.
"""
for dup in tqdm(list_of_duplicates):
    os.remove(dup)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 284/284 [00:00<00:00, 20764.24it/s]


In [80]:
print(len(os.listdir(IMG_DIR)))

2117
